In [5]:
import pandas as pd
import numpy as np

In [27]:
df_w5 = pd.read_stata('/Users/david.yan/Downloads/HCMST_wave_5_supplement_ver_1.dta')
df_w4 = pd.read_stata('/Users/david.yan/Downloads/wave_4_supplement_v1_2.dta')
df_w123 = pd.read_stata('/Users/david.yan/Downloads/HCMST_ver_3.04.dta')

In [28]:
w5_cols = df_w5.columns.tolist()
w4_cols = df_w4.columns.tolist()
w123_cols = df_w123.columns.tolist()

In [29]:
print(len(w5_cols), len(w4_cols), len(w123_cols))

(78, 62, 387)


In [34]:
print(df_w123.shape, df_w4.shape, df_w5.shape)

((4002, 387), (4002, 62), (4002, 78))


In [35]:
df_w123.isnull().sum()

caseid_new                        0
weight1                           0
weight2                         993
ppage                             0
ppagecat                          0
ppagect4                          0
ppeduc                            0
ppeducat                          0
ppethm                            0
ppgender                          0
pphhhead                          0
pphouseholdsize                   0
pphouse                           0
ppincimp                          0
hhinc                             0
ppmarit                           0
ppmsacat                          0
ppreg4                            0
ppreg9                            0
pprent                            0
ppt01                             0
ppt1317                           0
ppt18ov                           0
ppt25                             0
ppt612                            0
children_in_hh                    0
ppwork                            0
ppnet                       